In [ ]:
# getting query from user

def get_query():
    query = input("insert your query: ")
    query = clean(query)
    q = dict()
    oldnew = input("Do you prefer an old movie or a new-released movie? Please type O for Old and N for New:")
    if oldnew == "O" :
        q["release"] = "O"
    if oldnew == "N" :
        q["release"] = "N"



    year = input("Do you want to specify the release year ?Please type Y for Yes and N for No: ")
    if year == "N" :
        q["year"] = "NA"
    if year == "Y" :
        year = input("please specify the release date") 
        q["year"] = year



    Runtime = input("Do you want to specify the length of the movie? Please type Y for Yes and N for No:")
    if Runtime == "N" :
        q["Runtime"] = "NA"
    if Runtime == "Y" :
        Runtime = input("please specify the length of the movie")
        q["Runtime"] = Runtime


    starring = input("Is number of stars an important factor for you? Please type Y for Yes and N for No:")
    q["starring"] = starring


    budget = input("Is movie budget an important factor for you? Please type Y for Yes and N for No:")
    q["budget"] = budget
    
    return query,q

Now we have to run the first search engine with our query and get all the documents related to that search engine

In [ ]:
import re #don't forget to put it at first
import math

def search_engine3(query,q) :
    results = execute_query(query) #running the first search engine to get all query_related documents 
    # Now we should define variables that we want to use to give a new score
    
    result_variables = dict() # A dictionary that assigns each document to a dictionary of variables in that document
    score = dict() # A dictionary that assigns each document to the score of it
    for i in results :
        docId = i.split("_")[1] 
        with open(path2+'article_' + docId + '.tsv', encoding = 'utf8') as fd:
            rd = csv.reader(fd, delimiter="\t", quotechar='"')
            for row in rd:
                if row :
                    tsv = row
            

        d = {}

        if tsv[6] == 'NA':
            d['Starring'] = '0'
        else:
            d['Starring'] = str(len(tsv[6].split(',')))

        try:
            d['Release Year'] = re.search(r'\d{4}', tsv[8]).group(0)
        except:
            d['Release Year'] = '0'

        try:
            d['Runtime']      = re.search(r'\d+.*',tsv[9]).group(0)
        except:
            d['Runtime']    = '0'

        #some movies have running time expressed in reels, and the conversion in minutes is not univoque, so we'll just ignore those info
        if re.search(r'min', d['Runtime']):
            d['Runtime'] = re.search(r'\d+[\.|\,|:]*\d*', d['Runtime']).group(0)
            d['Runtime'] = re.search(r'\d+', d['Runtime']).group(0)
        else:
            d['Runtime'] = 0

        try:
            d['Budget']   = re.findall(r'\$.*', tsv[12])[0]
        except:
            d['Budget']  = '0'
            
        
        if re.search(r'mil', d['Budget']):
            d['Budget']  = str(int(float(re.search(r'\d+[\.|\,]*\d*', d['Budget']).group(0))*10**6))
            
        elif re.search(r',', d['Budget']):
            d['Budget'] = d['Budget'].replace(',', '').replace('$', '')
            
        else:
            d['Budget'] = d['Budget'].replace('.', '').replace('$', '')
             

        result_variables[docId] = d
        
    # Now we should define a function for each variable to assign a value between 0 and 1
        
    Runtimes = []
    Release_year = []
    Starring = []
    Budget = []

    for i in result_variables :
        Runtimes.append(d[i]["Runtime"])
        Release_year.append(d[i]["Release Year"])
        Starring.append(d[i]["Starring"])
        Budget.append(d[i]["Budget"])
        
        
    for i in range(len(result_variables)) :
        # calculating score for Running time
        minrun = min(Runtimes)
        maxrun = max(Runtimes)
        run_score = exp(-(int(q['Runtime'] -int(Runtimes[i]))**2)/100) 
        
        
        # calculting score for qualitative Release_year query 
        m = Release_year[i]
        maxx = max(Release_year)
        minn = min(Release_year)
        
        if (m != 0) :

            if (q["release"] == N) :
                release_score = 1 - (maxx - m)/(maxx - minn)


            if (q["release"] == O) :
                release_score = 1 - (m - minn) / (maxx - minn)
        
        else :
            release_score = 0
    
    
       # calculating score for quantitative Release_year query
        
        distance = abs(m - q["year"])
        year_score = math.exp(-distance/10)
        
        
      # calculating score for budget
    
    
    
    
     # calculating score for starring
        

        mean_score = 1/5 * (run_score + release_score + year_score + budget_score + starring_score)
        score[list(result_variables.keys())[i]] = mean_score
        

In [ ]:
heap = []
for i in score :
    heapq.heappush(heap,(score[i], i ))
heap_result = heapq.nlargest(10, heap)
df = pd.DataFrame({'Title': [], 'Intro': [], 'Url': [], 'Similarity' : []})
for x , y in heap_result :
    with open(path2+"article_" + str(y) + ".tsv", encoding = "utf-8") as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"')
        df.append({'Title': rd[0], 'Intro': rd[1], 'Url': movies[y], 'Similarity' : x}) # have many doubts about it :)
    